In [1]:
import requests
import datetime
import json
import gzip
import pandas as pd
import numpy as np
import os
import time
import logging

from datetime import datetime
from bs4 import BeautifulSoup
from io import StringIO

In [2]:
# Definimos el url de la web de conagua y un header con una identificación del navegador para evitar problemas de acceso
url = 'https://smn.conagua.gob.mx/webservices/?method=1'
header = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 \
          (KHTML, like Gecko) Mobile/15E148'}

# Se definen las rutas locales de donde se consumirán y se guardarán las tablas del ejercicio:

# En esta ruta se consume la data para cruzar
ruta_input = 'C:/Users/Usuario/Documents/data_municipios/'

# En esta ruta se guardan las tablas con los promedios por municipio 
ruta_tablas = 'C:/Users/Usuario/Documents/prueba_de/outputs/tablas/'

# En esta ruta se guarda la data de los promedios por municipio cruzada con la data más reciente
ruta_cruces = 'C:/Users/Usuario/Documents/prueba_de/outputs/tablas_cruzadas/'

# En esta ruta se guarda la copia con la data más reciente
ruta_current = 'C:/Users/Usuario/Documents/prueba_de/outputs/current/'

# En esta ruta se guarda el archivo con los logs
ruta_logs = 'C:/Users/Usuario/Documents/prueba_de/outputs/'

In [3]:
# Configuramos la ruta de guardado, el formato del log y con el modo de guardado en concatenación
logging.basicConfig(filename = f'{ruta_logs}logs.log',format = '%(asctime)s %(message)s',filemode = 'a')

In [4]:
def getData(url_1, header_1):
    
    ''' 
    Descarga datos en formato json comprimidos en gzip desde un servicio web, los procesa y los 
    retorna como un dataframe de Pandas.
    
    Parámetros
    ----------
    url_1: str
        URL del servicio web desde el cual se extraerá la data.   
    header_1: dict
        Encabezado HTTP de la solicitud. Puede incluir información como User-Agent, Content-Type, Date.
        
    Retorna
    -------
    data: dataframe
        Datos del servicio web estructurados.
        
    '''
    
    logger_scraping = logging.getLogger()
    
    try:
        result = requests.get(url_1, headers = header_1)
        data = gzip.decompress(result.content)
        data = json.loads(data)
        data = pd.DataFrame(data)
        data = data[['ides','idmun','prec','tmax','tmin']]

        data['ides'] = data['ides'].astype(int)
        data['idmun'] = data['idmun'].astype(int)
        data['prec'] = data['prec'].astype(float)
        data['tmax'] = data['tmax'].astype(float)
        data['tmin'] = data['tmin'].astype(float)
        
        logger_scraping.setLevel(logging.INFO)
        logger_scraping.info('Data extraída con éxito')

        return data
    
    except:
        logger_scraping.setLevel(logging.CRITICAL)
        logger_scraping.critical('Hubo un error al extraer o descomprimir la data')        

In [ ]:
# Número de veces que se repetirá el proceso.
n = 100

# Contadores
i = 0
count = 1

# Iteración que repite el proceso de extracción y manipulación periódicamente
while i <= n:
    
    # Se usa la función definida anteriomente para obtener la data desde el servicio web
    data = getData(url, header)
    data2 = pd.DataFrame()
    
    # Cada que se ejecuta el proceso, la data se concatena en un data frame. Cada segunda hora, los datos se agrupan
    # por municipio y se calculan los promedios. Posteriormente, la data se cruza y se guarda en una
    # carpeta local.
    
    # Si el módulo 2 del contador es 0 (es decir, cada que se cumplen 2 horas), se entra en esta condición
    if count % 2 == 0:

        data2 = pd.concat([data, data2], ignore_index = True)
        
        # Se agrupan los datos por municipio y se calculan promedios de precipitación y temperatura
        data_final = data2.groupby(['ides', 'idmun'], as_index = False).mean()
        data_final.rename(columns = {'ides':'Cve_Ent', 'idmun':'Cve_Mun', 'prec':'prec_avg',
                                    'tmax':'tmax_avg', 'tmin':'tmin_avg'}, inplace = True)
        data_final['tavg'] = data_final[['tmax_avg','tmin_avg']].mean(axis = 1)
        
        # Se determina la hora y la fecha actual
        now = datetime.now()
        now = now.strftime("%Y%m%d_%H%M")
        
        # Se guardan los datos agrupados versionados por fecha y hora
        data_final.to_csv(f'{ruta_tablas}{now}.csv')
    
        logger_data_avg = logging.getLogger()
        logger_data_avg.setLevel(logging.INFO)
        logger_data_avg.info('Data con promedios creada con éxito')
        
        # Se determina en automático cuál es la tabla más reciente en la carpeta rovista para realizar el 
        # cruce con los datos de temperatura y precipitación
        lista = os.listdir(ruta_input)
        lista_fechas = []

        for archivo in lista:

            anio = archivo[:4]
            mes = archivo[4:6]
            dia = archivo[6:8]

            date_archivo = pd.to_datetime(f'{anio}-{mes}-{dia}')
            date_archivo = date_archivo.date()

            lista_fechas.append(date_archivo)

        reciente = max(lista_fechas)

        id_reciente = int(str(reciente).split('-')[0] + str(reciente).split('-')[1] + str(reciente).split('-')[2])
        
        # Se lee la tabla más reciente y se realiza el cruce
        data_municipios = pd.read_csv(f'{ruta_input}/{id_reciente}/data.csv')
        data_municipios = data_municipios.merge(data_final, how = 'left')   
        data_municipios.fillna(0, inplace = True)

        # Se guardan versiones por fecha y hora en una carpeta y se crea la copia"current", que se actualiza cada vez
        # que se ejecuta esta condición
        
        data_municipios.to_csv(f'{ruta_cruces}/{now}.csv', index = False)
        data_municipios.to_csv(f'{ruta_current}/current.csv', index = False)
        
        logger_data_cruce = logging.getLogger()
        logger_data_cruce.setLevel(logging.INFO)
        logger_data_cruce.info('Data cruzada con éxito')

        del data, data2, data_final, data_municipios
        
        # Se detiene el proceso durante una hora
        time.sleep(3600)
        
    # Si el módulo 2 del contador es diferente de cero (es decir, cada primer hora), los datos extraídos simplemente
    # se concatenarán en un dataframe
    else: 
        data2 = pd.concat([data, data2], ignore_index = True)
        
        # Se detiene el roceso durante una hora
        time.sleep(3600)

    i += 1
    count += 1    